# MSC.Nastran DataTypes

Author : Emanuele Cannizzaro

In [1]:
import os
import sys
import xmltodict
import pandas as pd
from IPython.display import display, HTML
import xlsxwriter

In [2]:
ifname = os.path.join('..', 'docs', 'msc_datatype_2018.xml')

In [3]:
with open(ifname, 'r') as fd:
    doc = xmltodict.parse(fd.read())

Let's explore the data structure...

In [4]:
print(doc.keys())
print(doc['crdb'].keys())
#print(doc['crdb']['@schema'])
print(doc['crdb']['typedefs'].keys())
print(doc['crdb']['typedefs']['typedef'][0].keys())
print(doc['crdb']['typedefs']['typedef'][0]['field'][0].keys())
print(doc['crdb']['groups'].keys())
print(doc['crdb']['groups']['group'].keys())
if 0 == 1 :
    #doc['crdb']['groups']['group']['@name']
    for item in doc['crdb']['groups']['group']['group'] :
        print(item.keys())
        #print('{:20} -> {:-10} -> {:-10}'.format(item['@name'], item['group'], item['dataset']))
    for item in doc['crdb']['groups']['group']['group'] :
        print('{}'.format(item['@name']))
        print('{}'.format(item['group'][0].keys()))
        #print('{:20} -> {:-10} -> {:-10}'.format(item['@name'], item['group'], item['dataset'])))
    #print()

odict_keys(['crdb'])
odict_keys(['@schema', 'typedefs', 'groups'])
odict_keys(['typedef'])
odict_keys(['@name', 'field'])
odict_keys(['@name', '@type', '@description'])
odict_keys(['group'])
odict_keys(['@name', 'group'])


In [5]:
if 0 == 1:
    for item in doc['crdb']['typedefs']['typedef']:
        print('{:20} -> {:-10}'.format(item['@name'], len(item['field'])))

In [6]:
data = {}
data['Name'] = []
data['Field Name'] = []
data['Field Type'] = []
data['Field Description'] = []
for item in doc['crdb']['typedefs']['typedef'] :
    #print('{:20}'.format(item['@name']))
    if item['@name'] not in ['BEAM_SECTION', 'BEAM_SECTION3'] :
        for subitem in item['field'] :
            data['Name'].append(item['@name'])
            data['Field Name'].append(subitem['@name'])
            data['Field Type'].append(subitem['@type'])
            data['Field Description'].append(subitem['@description'])
typedefs = pd.DataFrame(data)
#display(HTML(typedefs.to_html()))

In [12]:
header = '''
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <meta name="description" content="">
    <meta name="author" content="">
    <link rel="icon" href="../../../../favicon.ico">

    <title>Cover Template for Bootstrap</title>

    <!-- Bootstrap core CSS -->
    <link href="bootstrap/css/bootstrap.min.css" rel="stylesheet">

    <!-- Custom styles for this template -->
    <link href="cover.css" rel="stylesheet">
  </head>

  <body class="text-center">

    <div class="cover-container d-flex w-100 h-100 p-3 mx-auto flex-column">
      <header class="masthead mb-auto">
        <div class="inner">
          <h3 class="masthead-brand">Cover</h3>
          <nav class="nav nav-masthead justify-content-center">
            <a class="nav-link active" href="#">Home</a>
            <a class="nav-link" href="#">Features</a>
            <a class="nav-link" href="#">Contact</a>
          </nav>
        </div>
      </header>

      <main role="main" class="inner cover">
        <h1 class="cover-heading">Nastran Typedefs</h1>
        <p class="lead"></p>
'''
tables = ''
bullets = '<ol>'
for name, group in typedefs.groupby('Name') :
        bullets += '<li><a href="#{0}">{0}</a></li>'.format(name)
        tables += '<h2 id="{0}">{0}</h2>'.format(name)
        tables += group.reset_index()[['Field Name', 'Field Type', 'Field Description']].to_html(escape=False)
#display(HTML(text))
bullets += '</ol>'

footer = '''
        <p class="lead">
          <a href="#" class="btn btn-lg btn-secondary">Learn more</a>
        </p>
      </main>

      <footer class="mastfoot mt-auto">
        <div class="inner">
          <p>Cover template for <a href="https://getbootstrap.com/">Bootstrap</a>, by <a href="">Emanuele Cannizzaro</a>.</p>
        </div>
      </footer>
    </div>


    <!-- Bootstrap core JavaScript
    ================================================== -->
    <!-- Placed at the end of the document so the pages load faster -->
    <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous"></script>
    <script>window.jQuery || document.write('<script src="../../assets/js/vendor/jquery-slim.min.js"><\/script>')</script>
    <script src="../../assets/js/vendor/popper.min.js"></script>
    <script src="../../dist/js/bootstrap.min.js"></script>
  </body>
</html>'''
with open(os.path.join('..', 'output', 'nastran.html'), 'w') as f :
    f.write(header)
    f.write(bullets)
    f.write(tables)

In [8]:
data = {}
data['Type'] = []
data['Entity'] = []
data['Keyword'] = []
data['Name'] = []
data['Field Name'] = []
data['Field Size'] = []
data['Field Type'] = []
data['Field Description'] = []
#print(len(doc['crdb']['groups']['group']['group']))
for item in doc['crdb']['groups']['group']['group'] :
    #print('{}'.format(item['@name']))
    #print(len(item['group']))
    for subitem in item['group'] :
        #print('\t{}'.format(subitem['@name']))
        if not((item['@name'] == 'INPUT') and \
               (subitem['@name'] in ['COORDINATE_SYSTEM', 'NODE'])) and \
        not((item['@name'] == 'RESULT') and \
            (subitem['@name'] in ['CONTACT', 'FATIGUE_VIBRATION', 'MODAL', 'SUMMARY', 'MONITOR'])) :
            #print(len(subitem['group']))
            for subsubitem in subitem['group'] :
                #print('\t\t{}'.format(subsubitem['@name']))
                if subsubitem['@name'] not in ['SPC1'] :
                    #print(len(subsubitem['dataset']))
                    if 'dataset' in subsubitem.keys() :
                        for subsubsubitem in subsubitem['dataset'] :
                            if isinstance(subsubsubitem, dict) :
                                #print('\t\t\t{}'.format(subsubsubitem['@name']))
                                if subsubsubitem['@name'] not in ['S', 'GIS', 'IDLIST', 'CPOINTS', 'GRIDS', 'HOMOCOORS', 'KNOTS', 
                                                                  'TRIMHOMOS', 'TRIMKNOTS'] :
                                    if 'field' in subsubsubitem.keys() :
                                        for subsubsubsubitem in subsubsubitem['field'] :
                                            #print(subsubsubsubitem.keys())
                                            if isinstance(subsubsubsubitem, dict) :
                                                data['Type'].append(item['@name'])
                                                data['Entity'].append(subitem['@name'])
                                                data['Keyword'].append(subsubitem['@name'])
                                                data['Name'].append(subsubsubitem['@name'])
                                                if '@name' in subsubsubsubitem.keys() :
                                                    data['Field Name'].append(subsubsubsubitem['@name'])
                                                else :
                                                    data['Field Name'].append(None)
                                                if '@type' in subsubsubsubitem.keys() :
                                                    data['Field Type'].append(subsubsubsubitem['@type'])
                                                else :
                                                    data['Field Type'].append(None)
                                                if '@size' in subsubsubsubitem.keys() :
                                                    data['Field Size'].append(subsubsubsubitem['@size'])
                                                else :
                                                    data['Field Size'].append(None)
                                                if '@description' in subsubsubsubitem.keys() :
                                                    data['Field Description'].append(subsubsubsubitem['@description'])
                                                else :
                                                    data['Field Description'].append(None)
#                for subitem in subitem['field'])) :
#                    data['Name'].append(subitem['dataset'][ssid]['@name'])                    
#                    data['Field Name'] = [subitem['dataset'][_]['field'] for _ in range(n)]
#                    data['Field Type'] = [subitem['dataset'][_]['field'] for _ in range(n)]
#                    data['Field Description'] = [subitem['dataset'][_]['field'] for _ in range(n)]
                    #data['Description'] = [subitem['group'][_] for _ in range(n)]
#                    typedefs[subitem['@name']] = pd.DataFrame(data)
#                    display(HTML(typedefs[subitem['@name']].to_html()))
#            print()
#                data['Dataset'] = item['group'][id]['dataset']
#                print(data['Dataset'])
groups = pd.DataFrame(data)

In [9]:
#display(HTML(df.to_html()))

In [10]:
groups

,Type,Entity,Keyword,Name,Field Name,Field Size,Field Type,Field Description
0,INPUT,CONSTRAINT,AELINK,IDENTITY,ID,None,integer,If >0 TRIM set ID selected in Case Control. If...
1,INPUT,CONSTRAINT,AELINK,IDENTITY,LABLD,8,character,The dependent aerodynamic variable
2,INPUT,CONSTRAINT,AELINK,IDENTITY,TERMS_POS,None,integer,Start position in dataset TERMS
3,INPUT,CONSTRAINT,AELINK,IDENTITY,TERMS_LEN,None,integer,Number of rows in dataset TERMS
4,INPUT,CONSTRAINT,AELINK,IDENTITY,DOMAIN_ID,None,integer,None
5,INPUT,CONSTRAINT,AELINK,TERMS,LABLI,8,character,I-th independent aerodynamic variable
6,INPUT,CONSTRAINT,AELINK,TERMS,CI,None,double,Linking coefficient for i-th variable
7,INPUT,CONSTRAINT,MPC,GCA,G,None,integer,None
8,INPUT,CONSTRAINT,MPC,GCA,C,None,integer,None
9,INPUT,CONSTRAINT,MPC,GCA,A,None,double,None


In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(os.path.join('..', 'output', 'nastran.xlsx'), engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
typedefs.to_excel(writer, sheet_name='typedefs')
groups.to_excel(writer, sheet_name='groups')

# Close the Pandas Excel writer and output the Excel file.
writer.save()